In [1]:
import pandas as pd

# Merge all the daily data for the funnel
csv_directory = '../CSV/'
merged_data = pd.DataFrame()

# List of file names
file_names = ["funneldatadaily1.csv", "funneldatadaily2.csv", "funneldatadaily3.csv", "funneldatadaily4.csv",
              "funneldatadaily5.csv", "funneldatadaily6.csv", "funneldatadaily7.csv", "funneldatadaily8.csv",
              "funneldatadaily9.csv", "funneldatadaily10.csv", "funneldatadaily11.csv", "funneldatadaily12.csv",
              "funneldatadaily13.csv"]

# Loop through each file and merge it into the DataFrame
for file_name in file_names:
    df = pd.read_csv(csv_directory + file_name)
    merged_data = pd.concat([merged_data, df], ignore_index=True)

# Specify the path to save the merged file
merged_file_path = '../CSV/fulldatadaily.csv'

# Save the merged data to a new CSV file
merged_data.to_csv(merged_file_path, index=False)



In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interactive, Layout, Output
from IPython.display import display, clear_output
from datetime import datetime


# Create date pickers for the date range filter
start_date_picker = widgets.DatePicker(
    description='Start Date:',
    value=pd.Timestamp('2021-01-01'),
    disabled=False
)
end_date_picker = widgets.DatePicker(
    description='End Date:',
    value=pd.Timestamp('2022-04-24'),
    disabled=False
)
# Add a checkbox widget for the "Merging" option
total_checkbox = widgets.Checkbox(
    description='Merge Data',
    value=False, 
    disabled=False
)   

# Create a function to display a plot
def display_download_platform():
    # Load the dataset
    df = pd.read_csv('../CSV/funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 1]
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range
    def filter_data(start_date, end_date, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
        (df['daily_date'].dt.date >= start_date) & 
        (df['daily_date'].dt.date <= end_date)  & 
        (df['platform'] >= start_platform) &
        (df['platform'] <= end_platform)]
        return filtered_df

    # Define the default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=df['platform'].unique(),
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=df['platform'].unique(),
        value=default_end_platform_range,
        disabled=False
    )    
    
# Update function for the line plot
    def update_line_plot(start_date, end_date, start_platform, end_platform, total):
        if total:
        # If the "Total" checkbox is checked, calculate the total users by date
            filtered_data = filter_data(start_date, end_date, start_platform, end_platform)
            total_data = filtered_data.groupby('daily_date')['users'].sum().reset_index()

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)  
            
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=total_data['daily_date'], y=total_data['users'], name='Total'))
        else:
        # If "Total" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_platform, end_platform)

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)            
            fig = go.Figure()

            platforms = filtered_data['platform'].unique()

            for platform in platforms:
                platform_data = filtered_data[filtered_data['platform'] == platform]
                platform_data_grouped = platform_data.groupby('daily_date')['users'].sum().reset_index()

                fig.add_trace(go.Scatter(x=platform_data_grouped['daily_date'], y=platform_data_grouped['users'], name=platform))

        fig.update_layout(
                            title="Daily Downloads per Platform",
                            xaxis_title="Date",
                            yaxis_title="Downloads",
                            width=width_pixels,
                            height=height_pixels,            
                         )

        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_download_platform.html')
        
        fig.show()

    # Create an interactive widget to update the line plot
    widget = interactive(
        update_line_plot, 
        start_date=start_date_picker, 
        end_date=end_date_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)
    




In [3]:
def display_funnel_users_visualization():
    # Your existing code for the funnel visualization
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import plotly.graph_objects as go
    import ipywidgets as widgets
    from ipywidgets import interactive, Layout, Output
    from IPython.display import display, clear_output
    from datetime import datetime
    
    df = pd.read_csv('../CSV/fulldatadaily.csv')
    # Define the date range selection widgets
    start_date_picker = widgets.DatePicker(
        description='Start Date:',
        value=pd.Timestamp('2021-01-01'),
        disabled=False
    )

    end_date_picker = widgets.DatePicker(
        description='End Date:',
        value=pd.Timestamp('2022-04-24'),
        disabled=False
    )

    # Define your default age ranges
    default_start_age_range = 'None'
    default_end_age_range = 'Unknown'

    # Filter out 'Unknown' and NaN values from the age_range column
    filtered_age_ranges = df[df['age_range'].notna() & (df['age_range'] != 'Unknown')]['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: (float(x.split('-')[0]) if '-' in x else float('inf')))
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')
    unique_age_ranges.append('None')

    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )

    # Sort the platform options alphabetically and drop nan values
    platform_options = sorted(df['platform'].dropna().astype(str).unique())

    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'

    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=platform_options,
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=platform_options,
        value=default_end_platform_range,
        disabled=False
    )

    def display_funnel_users_visualization(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        df = pd.read_csv('../CSV/fulldatadaily.csv')
        df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')
        start_date = pd.Timestamp(start_date)
        end_date = pd.Timestamp(end_date)
        if start_age_range != 'None' and end_age_range != 'None':
            df = df[(df['age_range'] >= start_age_range) & (df['age_range'] <= end_age_range)]
        df = df[(df['daily_date'] >= start_date) & (df['daily_date'] <= end_date) &
                (df['platform'] >= start_platform) & (df['platform'] <= end_platform)]
        funnel_data = df.groupby(['funnel_step', 'funnel_name', 'platform']).agg({
            'user_ids': 'nunique'
        }).reset_index()
        name_correction_dict = {
            "downloads": "Downloads",
            "signups": "Signups",
            "rides_requested": "Rides Requested",
            "rides_completed": "Rides Completed",
        }        
        funnel_data = funnel_data.sort_values(by="funnel_step", ascending=False)
        # Apply name corrections to the 'funnel_name' column
        funnel_data['funnel_name'] = funnel_data['funnel_name'].replace(name_correction_dict)        
        pivot_df = funnel_data.pivot(index="funnel_step", columns="platform", values="user_ids")
        pivot_df_names = funnel_data.pivot(index="funnel_step", columns="platform", values="funnel_name")
        colors = {
            "android": "rgb(0, 123, 255)",
            "ios": "rgb(40, 167, 69)",
            "web": "rgb(220, 53, 69)",
        }
        figsize_inches =  (8.27, 5.87)
        dpi = 96
        width_pixels = int(figsize_inches[0] * dpi)
        height_pixels = int(figsize_inches[1] * dpi)
        fig = go.Figure()
        for platform in pivot_df.columns:
            fig.add_trace(go.Funnel(
                #add labels to the graphic
                name=platform,
                y=pivot_df_names[platform],
                x=pivot_df[platform],
                textinfo="value+percent initial",
                textposition="inside",
                opacity=0.75,
                marker=dict(color=colors[platform]),
                #texttemplate='%{value}',
            ))
        fig.update_layout(
            title="User Funnel",
            legend_title="Platform",
            width=width_pixels,
            height=height_pixels,
        )
        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_users_funnel.html')          
        fig.show()

    # Create an interactive widget to update the funnel chart
    interactive_plot = interactive(
        display_funnel_users_visualization,
        start_date=start_date_picker,
        end_date=end_date_picker,
        start_age_range=start_age_range_picker,
        end_age_range=end_age_range_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker
    )

    # Display the interactive widget
    output = interactive_plot.children[-1]
    output.layout.height = '400px'
    display(interactive_plot)
  
# Create an output widget for displaying the visualizations
output = widgets.Output()


In [4]:
def display_funnel_rides_visualization():
    import numpy as np
    import pandas as pd
    import plotly.graph_objects as go
    import ipywidgets as widgets
    from ipywidgets import interactive, Layout, Output
    from IPython.display import display, clear_output
    from datetime import datetime
    
    # Load your data (replace 'data.csv' with your data file)
    df = pd.read_csv('../CSV/fulldatadaily.csv')

    # Define filtering options for age range, date, and platform
    start_date_picker = widgets.DatePicker(
        description='Start Date:',
        value=pd.Timestamp('2021-01-01'),
        disabled=False
    )

    end_date_picker = widgets.DatePicker(
        description='End Date:',
        value=pd.Timestamp('2022-04-24'),
        disabled=False
    )

    # Define your default age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'

    # Filter out 'Unknown' and NaN values from the age_range column
    filtered_age_ranges = df[df['age_range'].notna() & (df['age_range'] != 'Unknown')]['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: (float(x.split('-')[0]) if '-' in x else float('inf')))

    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')
    unique_age_ranges.append('None')

    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )

    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )

    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'

    # Sort the platform options alphabetically and drop nan values
    platform_options = sorted(df['platform'].dropna().astype(str).unique())

    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=platform_options,
        value=default_start_platform_range,
        disabled=False
    )

    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=platform_options,
        value=default_end_platform_range,
        disabled=False
    )

    # Define a function to display the funnel chart
    def display_funnel_rides_visualization(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):

        df = pd.read_csv('../CSV/fulldatadaily.csv')
        # Filter data based on selected filters
        df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')
        start_date = pd.Timestamp(start_date)
        end_date = pd.Timestamp(end_date)

        filtered_df = df[
            (df['funnel_step'].isin([3, 4])) &  # Filter for funnel steps 3 and 4
            (df['daily_date'] >= pd.Timestamp(start_date)) &
            (df['daily_date'] <= pd.Timestamp(end_date)) &
            ((df['age_range'] >= start_age_range) | (df['age_range'].isna())) &
            ((df['age_range'] <= end_age_range) | (df['age_range'] == 'Unknown')) &
            (df['platform'] >= start_platform) &
            (df['platform'] <= end_platform)
        ]

        # Group and aggregate data for the funnel
        funnel_data = filtered_df.groupby(['funnel_step', 'funnel_name', 'platform']).agg({
            'user_ids': 'nunique',  # Count unique user IDs
            'rides': 'sum'  # Sum of rides
        }).reset_index()

        name_correction_dict = {
            "rides_requested": "Rides Requested",
            "rides_completed": "Rides Completed",
        }          
        # Sort the DataFrame by funnel step in ascending order
        funnel_data = funnel_data.sort_values(by="funnel_step")
        # Apply name corrections to the 'funnel_name' column
        funnel_data['funnel_name'] = funnel_data['funnel_name'].replace(name_correction_dict) 
        # Pivot the data for plotting
        pivot_df = funnel_data.pivot(index="funnel_step", columns="platform", values="rides")
        pivot_df_names = funnel_data.pivot(index="funnel_step", columns="platform", values="funnel_name")

        # Specify colors for each platform
        colors = {
            "android": "rgb(0, 123, 255)",
            "ios": "rgb(40, 167, 69)",
            "web": "rgb(220, 53, 69)",
        }

        # Create a Plotly funnel chart with specified colors
        figsize_inches = (8.27, 5.87)
        dpi = 96
        width_pixels = int(figsize_inches[0] * dpi)
        height_pixels = int(figsize_inches[1] * dpi)
        fig = go.Figure()

        # Group the data by platform
        for platform in pivot_df.columns:
            fig.add_trace(go.Funnel(
                name=platform,
                y=pivot_df_names[platform],
                x=pivot_df[platform],
                textinfo="value+percent initial",
                textposition="inside",
                opacity=0.75,
                marker=dict(color=colors[platform]),
            ))

        # Set the layout
        fig.update_layout(
            title="Rides Funnel",
            legend_title="Platform",
            width=width_pixels,
            height=height_pixels,
        )

        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_rides_funnel.html')  
    
        # Show the Plotly chart
        fig.show()

    # Create an interactive widget for the funnel chart
    interactive_funnel = interactive(
        display_funnel_rides_visualization,
        start_date=start_date_picker,
        end_date=end_date_picker,
        start_age_range=start_age_range_picker,
        end_age_range=end_age_range_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker
    )

    # Display the interactive widget
    display(interactive_funnel)
    
# Create an output widget for displaying the visualizations
output = widgets.Output()

In [5]:
def display_signups_visualization():
    # Load the dataset
    df = pd.read_csv('../CSV/funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 2]
    
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range and age range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
            (df['daily_date'].dt.date >= start_date) &
            (df['daily_date'].dt.date <= end_date) &
            (df['age_range'] >= start_age_range) &
            (df['age_range'] <= end_age_range) &
            (df['platform'] >= start_platform) &
            (df['platform'] <= end_platform)
        ]
        return filtered_df
    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=df['platform'].unique(),
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=df['platform'].unique(),
        value=default_end_platform_range,
        disabled=False
    )

    # Update function for the bar chart
    def update_bar_chart(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Merge" checkbox is checked, calculate the total users
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by=["age_range", "platform"], ascending=[True, True])            
            total_data = filtered_data.groupby(['platform'])['users'].sum().reset_index()

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)  
            
            fig = go.Figure()
            fig.add_trace(go.Bar(x=total_data['platform'], y=total_data['users'], name='Total'))
        else:
            # If "Merge" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by=["age_range", "platform"], ascending=[True, True])  

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)            
            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
            age_ranges = filtered_data['age_range'].unique()

            for age_range in age_ranges:
                age_range_data = filtered_data[filtered_data['age_range'] == age_range]
                users_by_platform = []

                for platform in platforms:
                    platform_data = age_range_data[age_range_data['platform'] == platform]
                    users = platform_data['users'].sum()
                    users_by_platform.append(users)

                # Create a label that includes both age_range and platform information
                label = [f'{age_range} - {platform}' for platform in platforms]

                fig.add_trace(go.Bar(x=label, y=users_by_platform, name=age_range))

        fig.update_layout(
                            title="Signups by Age Range - Platform",
                            xaxis_title="Age Range - Platform",
                            yaxis_title="Signups",
                            width=width_pixels,
                            height=height_pixels,            
                        )

        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_signups.html')  

        fig.show()
    # Create an interactive widget to update the bar chart
    widget = interactive(
        update_bar_chart, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)



In [6]:
# Create a function to display a plot
def display_signupsdaily_visualization():
    # Load the dataset
    df = pd.read_csv('../CSV/funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 2]
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
        (df['daily_date'].dt.date >= start_date) & 
        (df['daily_date'].dt.date <= end_date) &
        (df['age_range'] >= start_age_range) & 
        (df['age_range'] <= end_age_range) &        
        (df['platform'] >= start_platform) &
        (df['platform'] <= end_platform)
        ]
        return filtered_df

    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )    
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
        
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=df['platform'].unique(),
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=df['platform'].unique(),
        value=default_end_platform_range,
        disabled=False
    )

# Update function for the line plot
    def update_line_plot(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
        # If the "Merge" checkbox is checked, calculate the total users by date
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            total_data = filtered_data.groupby('daily_date')['users'].sum().reset_index()
            
            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=total_data['daily_date'], y=total_data['users'], name='Total'))
        else:
        # If "Merge" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            
            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            fig = go.Figure()

            platforms = filtered_data['platform'].unique()

            for platform in platforms:
                platform_data = filtered_data[filtered_data['platform'] == platform]
                platform_data_grouped = platform_data.groupby('daily_date')['users'].sum().reset_index()

                fig.add_trace(go.Scatter(x=platform_data_grouped['daily_date'], y=platform_data_grouped['users'], name=platform))

        fig.update_layout(
                            title="Daily Signups per Platform",
                            xaxis_title="Date",
                            yaxis_title="Signups",
                            width=width_pixels,
                            height=height_pixels,            
                        )

        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_signups_daily.html')          
        fig.show()

    # Create an interactive widget to update the line plot
    widget = interactive(
        update_line_plot, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,        
        start_platform=start_platform_range_picker, 
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)
    


In [7]:
def display_user_by_riderequests_visualization():
    # Load the dataset
    df = pd.read_csv('../CSV/fulldatadaily.csv')
    df = df[df['funnel_step'] == 3]
    
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range and age range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
            (df['daily_date'].dt.date >= start_date) &
            (df['daily_date'].dt.date <= end_date) &
            (df['age_range'] >= start_age_range) &
            (df['age_range'] <= end_age_range) &
            (df['platform'] >= start_platform) &
            (df['platform'] <= end_platform)
        ]
        return filtered_df

    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
    #Get unique platform options and sort them
    platform_options = sorted(df['platform'].dropna().astype(str).unique())
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=platform_options,
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=platform_options,
        value=default_end_platform_range,
        disabled=False
    )
    # Update function for the bar chart
    def update_bar_chart(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Merge" checkbox is checked, calculate the total users
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by=["age_range", "platform"], ascending=[True, True])


            # Define a custom aggregation function to count unique values
            def unique_count(series):
                return series.nunique()

            total_data = filtered_data.groupby(['platform']).agg({
                'user_ids': unique_count,
                'rides': 'sum'
            }).reset_index()            
            
            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()
            fig.add_trace(go.Bar(x=total_data['platform'], y=total_data['user_ids'], name='Total Users'))
        else:
            # If "Merge" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by=["age_range", "platform"], ascending=[True, True])           

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
            age_ranges = filtered_data['age_range'].unique()

            for age_range in age_ranges:
                age_range_data = filtered_data[filtered_data['age_range'] == age_range]
                users_by_platform = []

                for platform in platforms:
                    platform_data = age_range_data[age_range_data['platform'] == platform]
                    unique_users = platform_data['user_ids'].nunique()
                    users_by_platform.append(unique_users)

                # Create a label that includes both age_range and platform information
                label = [f'{age_range} - {platform}' for platform in platforms]

                fig.add_trace(go.Bar(x=label, y=users_by_platform, name=age_range))

        fig.update_layout(
                            title="Ride Requested Users by Age Range - Platform",
                            xaxis_title="Age Range - Platform",
                            yaxis_title="Users",
                            width=width_pixels,
                            height=height_pixels,
                        )

        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_users_by_rides_requested.html')  
        fig.show()
    # Create an interactive widget to update the bar chart
    widget = interactive(
        update_bar_chart, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)


In [8]:
def display_riderequests_visualization():
    # Load the dataset
    df = pd.read_csv('../CSV/funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 3]
    
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range and age range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
            (df['daily_date'].dt.date >= start_date) &
            (df['daily_date'].dt.date <= end_date) &
            (df['age_range'] >= start_age_range) &
            (df['age_range'] <= end_age_range) &
            (df['platform'] >= start_platform) &
            (df['platform'] <= end_platform)
        ]
        return filtered_df

    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
    #Get unique platform options and sort them
    platform_options = sorted(df['platform'].dropna().astype(str).unique())
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=platform_options,
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=platform_options,
        value=default_end_platform_range,
        disabled=False
    )
    # Update function for the bar chart
    def update_bar_chart(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Merge" checkbox is checked, calculate the total users
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by=["age_range", "platform"], ascending=[True, True])
            total_data = filtered_data.groupby(['platform'])['rides'].sum().reset_index()
            
            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()
            fig.add_trace(go.Bar(x=total_data['platform'], y=total_data['rides'], name='Total'))
        else:
            # If "Merge" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by=["age_range", "platform"], ascending=[True, True])

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
            age_ranges = filtered_data['age_range'].unique()

            for age_range in age_ranges:
                age_range_data = filtered_data[filtered_data['age_range'] == age_range]
                rides_by_platform = []

                for platform in platforms:
                    platform_data = age_range_data[age_range_data['platform'] == platform]
                    rides = platform_data['rides'].sum()
                    rides_by_platform.append(rides)

                # Create a label that includes both age_range and platform information
                label = [f'{age_range} - {platform}' for platform in platforms]

                fig.add_trace(go.Bar(x=label, y=rides_by_platform, name=age_range))

        fig.update_layout(
                            title="Rides Requested by Age Range - Platform",
                            xaxis_title="Age Range - Platform",
                            yaxis_title="Rides Requested",
                            width=width_pixels,
                            height=height_pixels,
                        )

        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_ride_requests.html')  
        fig.show()
    # Create an interactive widget to update the bar chart
    widget = interactive(
        update_bar_chart, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)



In [9]:
# Create a function to display a plot
def display_rides_cancellations_visualization():
    # Load the dataset
    df = pd.read_csv('../CSV/funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 3]
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
        (df['daily_date'].dt.date >= start_date) & 
        (df['daily_date'].dt.date <= end_date) &
        (df['age_range'] >= start_age_range) & 
        (df['age_range'] <= end_age_range) &        
        (df['platform'] >= start_platform) &
        (df['platform'] <= end_platform)
        ]
        return filtered_df
    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )    
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
    #Sort platform and drop nan values
    platform_options = sorted(df['platform'].dropna().astype(str).unique())        
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=platform_options,
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=platform_options,
        value=default_end_platform_range,
        disabled=False
    )

    # Update function for the line plot
    def update_line_plot(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Merge Platforms" checkbox is checked, calculate the total rides and cancellations
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by="platform", ascending=True)
            total_rides = filtered_data.groupby('daily_date')['rides'].sum().reset_index()
            total_cancellations = filtered_data.groupby('daily_date')['cancellations'].sum().reset_index()

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=total_rides['daily_date'], y=total_rides['rides'], name='Rides'))
            fig.add_trace(go.Scatter(x=total_cancellations['daily_date'], y=total_cancellations['cancellations'], name='Cancellations'))
        else:
            # If "Merge Platforms" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by="platform", ascending=True)
            
            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
        
            for platform in platforms:
                platform_data = filtered_data[filtered_data['platform'] == platform]
                platform_data_grouped = platform_data.groupby('daily_date')[['rides', 'cancellations']].sum().reset_index()
                fig.add_trace(go.Scatter(x=platform_data_grouped['daily_date'], y=platform_data_grouped['rides'], name=f'{platform} Rides'))
                fig.add_trace(go.Scatter(x=platform_data_grouped['daily_date'], y=platform_data_grouped['cancellations'], name=f'{platform} Cancellations'))
    
        fig.update_layout(
                            title="Daily Rides & Cancellations per Platform",
                            xaxis_title="Date",
                            yaxis_title="Count",
                            width=width_pixels,
                            height=height_pixels,
                         )
    
         # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_rides_cancellations.html')             
        
        fig.show()

    # Create an interactive widget to update the line plot
    widget = interactive(
        update_line_plot, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,        
        start_platform=start_platform_range_picker, 
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)



In [10]:
def display_surge_pricing():
    # Load the dataset from the CSV file
    df = pd.read_csv('../CSV/surge_pricing.csv')

    # Create a grouped bar chart
    fig = go.Figure()
    
    # Calculate the width and height in pixels based on the desired figsize in inches and DPI
    figsize_inches = (8.27, 5.87)
    dpi = 96
    width_pixels = int(figsize_inches[0] * dpi)
    height_pixels = int(figsize_inches[1] * dpi)
    
    # Define the unique requests values
    hours = df['request_hour']
    num_requests = df['num_requests']

    fig.add_trace(go.Bar(
        x=hours,
        y=num_requests,
        hoverinfo='x+y',
        text=num_requests,
        textposition='auto',
        marker=dict(color='blue')
    ))

    # Update the layout
    fig.update_layout(
        title='Surge-Pricing',
        xaxis_title='Hour',
        yaxis_title='Ride Requests',
        xaxis=dict(dtick=1),
        showlegend=False,
        width=width_pixels,
        height=height_pixels
    )

    # Transform the interactive Plotly chart into an Html and save it.
    fig.write_html('../InteractiveGraphics/interactive_surge_pricing.html')
    
    # Show the chart
    fig.show()


In [11]:
def display_minridesperuser():
    # Load the dataset from the CSV file
    df = pd.read_csv('../CSV/minridesperuser.csv')

    figsize_inches = (8.27, 5.87)
    dpi = 96
    width_pixels = int(figsize_inches[0] * dpi)
    height_pixels = int(figsize_inches[1] * dpi)
    
    # Create a grouped bar chart
    fig = go.Figure()

    # Define the unique charge_status values
    user_statuses = df['user_id'].unique()
    bins = list(range(0, 61, 5))

    # Create a list to store the counts of unique users for each 'rides' range
    user_counts = []

    for bin_start, bin_end in zip(bins[:-1], bins[1:]):
        user_count = df[(df['rides'] >= bin_start) & (df['rides'] < bin_end)]['user_id'].nunique()
        user_counts.append(user_count)

    fig.add_trace(go.Bar(
        y=user_counts,
        x=[f"{bin_start}-{bin_end}" for bin_start, bin_end in zip(bins[:-1], bins[1:])],
        hoverinfo='x+y',
        text=user_counts,
        textposition='auto',
    ))

    # Update the layout
    fig.update_layout(
        title='Minimum Rides Per User',
        xaxis_title='Number of Rides(Range)',
        yaxis_title='Number of Users',
        barmode='group',
        width=width_pixels,
        height=height_pixels,        
    )
    # Transform the interactive Plotly chart into an Html and save it.
    fig.write_html('../InteractiveGraphics/interactive_minridesperuser.html')
    # Show the chart
    fig.show()


In [12]:
def display_user_by_ridecompletes_visualization():
    # Load the dataset
    df = pd.read_csv('../CSV/fulldatadaily.csv')
    df = df[df['funnel_step'] == 4]
    
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range and age range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
            (df['daily_date'].dt.date >= start_date) &
            (df['daily_date'].dt.date <= end_date) &
            (df['age_range'] >= start_age_range) &
            (df['age_range'] <= end_age_range) &
            (df['platform'] >= start_platform) &
            (df['platform'] <= end_platform)
        ]
        return filtered_df

    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
    #Get unique platform options and sort them
    platform_options = sorted(df['platform'].dropna().astype(str).unique())
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=platform_options,
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=platform_options,
        value=default_end_platform_range,
        disabled=False
    )
    # Update function for the bar chart
    def update_bar_chart(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Merge" checkbox is checked, calculate the total users
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by=["age_range", "platform"], ascending=[True, True])


            # Define a custom aggregation function to count unique values
            def unique_count(series):
                return series.nunique()

            total_data = filtered_data.groupby(['platform']).agg({
                'user_ids': unique_count,
                'rides': 'sum'
            }).reset_index()            
            
            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()
            fig.add_trace(go.Bar(x=total_data['platform'], y=total_data['user_ids'], name='Total Users'))
        else:
            # If "Merge" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by=["age_range", "platform"], ascending=[True, True])           

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
            age_ranges = filtered_data['age_range'].unique()

            for age_range in age_ranges:
                age_range_data = filtered_data[filtered_data['age_range'] == age_range]
                users_by_platform = []

                for platform in platforms:
                    platform_data = age_range_data[age_range_data['platform'] == platform]
                    unique_users = platform_data['user_ids'].nunique()
                    users_by_platform.append(unique_users)

                # Create a label that includes both age_range and platform information
                label = [f'{age_range} - {platform}' for platform in platforms]

                fig.add_trace(go.Bar(x=label, y=users_by_platform, name=age_range))

        fig.update_layout(
                            title="Rides Completed Users by Age Range - Platform",
                            xaxis_title="Age Range - Platform",
                            yaxis_title="Users",
                            width=width_pixels,
                            height=height_pixels,
                        )

        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_users_by_rides_completed.html')  
        fig.show()
    # Create an interactive widget to update the bar chart
    widget = interactive(
        update_bar_chart, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)


In [13]:
def display_completed_age_platform():
    # Load the dataset
    df = pd.read_csv('../CSV/funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 4]
    
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range and age range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
            (df['daily_date'].dt.date >= start_date) &
            (df['daily_date'].dt.date <= end_date) &
            (df['age_range'] >= start_age_range) &
            (df['age_range'] <= end_age_range) &
            (df['platform'] >= start_platform) &
            (df['platform'] <= end_platform)
        ]
        return filtered_df

    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'

    #Sort platform and drop nan values
    platform_options = sorted(df['platform'].dropna().astype(str).unique())    
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=platform_options,
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=platform_options,
        value=default_end_platform_range,
        disabled=False
    )
    # Update function for the bar chart
    def update_bar_chart(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Merge" checkbox is checked, calculate the total users
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by=["age_range", "platform"], ascending=[True, True])

            total_data = filtered_data.groupby(['platform'])['rides'].sum().reset_index()

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()
            fig.add_trace(go.Bar(x=total_data['platform'], y=total_data['rides'], name='Total'))
        else:
            # If "Merge" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by=["age_range", "platform"], ascending=[True, True])

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
            age_ranges = filtered_data['age_range'].unique()

            for age_range in age_ranges:
                age_range_data = filtered_data[filtered_data['age_range'] == age_range]
                rides_by_platform = []

                for platform in platforms:
                    platform_data = age_range_data[age_range_data['platform'] == platform]
                    rides = platform_data['rides'].sum()
                    rides_by_platform.append(rides)

                # Create a label that includes both age_range and platform information
                label = [f'{age_range} - {platform}' for platform in platforms]

                fig.add_trace(go.Bar(x=label, y=rides_by_platform, name=age_range))

        fig.update_layout(
                            title="Rides Completed by Age Range - Platform",
                            xaxis_title="Age Range - Platform",
                            yaxis_title="Rides",
                            width=width_pixels,
                            height=height_pixels,            
                         )

        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_completed_age_platform.html')         

        fig.show()
    # Create an interactive widget to update the bar chart
    widget = interactive(
        update_bar_chart, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)
    


In [14]:
def display_completed_usd():
    # Load the dataset
    df = pd.read_csv('../CSV/funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 4]
    
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range and age range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
            (df['daily_date'].dt.date >= start_date) &
            (df['daily_date'].dt.date <= end_date) &
            (df['age_range'] >= start_age_range) &
            (df['age_range'] <= end_age_range) &
            (df['platform'] >= start_platform) &
            (df['platform'] <= end_platform)
        ]
        return filtered_df
  
    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'
    
    #Sort platform and drop nan values
    platform_options = sorted(df['platform'].dropna().astype(str).unique())  
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=platform_options,
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=platform_options,
        value=default_end_platform_range,
        disabled=False
    )

    # Update function for the bar chart
    def update_bar_chart(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Merge" checkbox is checked, calculate the total users
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by=["age_range", "platform"], ascending=[True, True])
        
            total_data = filtered_data.groupby(['platform'])['total_usd'].sum().reset_index()

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()
            fig.add_trace(go.Bar(x=total_data['platform'], y=total_data['total_usd'], name='USD'))
        else:
            # If "Merge" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by=["age_range", "platform"], ascending=[True, True])

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
            age_ranges = filtered_data['age_range'].unique()

            for age_range in age_ranges:
                age_range_data = filtered_data[filtered_data['age_range'] == age_range]
                usd_by_platform = []

                for platform in platforms:
                    platform_data = age_range_data[age_range_data['platform'] == platform]
                    usd = platform_data['total_usd'].sum()
                    usd_by_platform.append(usd)

                # Create a label that includes both age_range and platform information
                label = [f'{age_range} - {platform}' for platform in platforms]

                fig.add_trace(go.Bar(x=label, y=usd_by_platform, name=age_range))

        fig.update_layout(
                            title="USD by Age Range - Platform",
                            xaxis_title="Age Range - Platform",
                            yaxis_title="USD",
                            width=width_pixels,
                            height=height_pixels,            
                         )

        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_completed_usd.html')            
        
        fig.show()
    # Create an interactive widget to update the bar chart
    widget = interactive(
        update_bar_chart, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)


In [15]:
# Create a function to display a plot
def display_completed_usd_daily():
    # Load the dataset
    df = pd.read_csv('../CSV/funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 4]
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
        (df['daily_date'].dt.date >= start_date) & 
        (df['daily_date'].dt.date <= end_date) &
        (df['age_range'] >= start_age_range) & 
        (df['age_range'] <= end_age_range) &        
        (df['platform'] >= start_platform) &
        (df['platform'] <= end_platform)
        ]
        return filtered_df

    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )    
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'

    platform_options = sorted(df['platform'].dropna().astype(str).unique())  
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=platform_options,
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=platform_options,
        value=default_end_platform_range,
        disabled=False
    )

    # Update function for the line plot
    def update_line_plot(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Merge Platforms" checkbox is checked, calculate the total rides and cancellations
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by="platform", ascending=True)            
            total_rides = filtered_data.groupby('daily_date')['rides'].sum().reset_index()
            total_usd = filtered_data.groupby('daily_date')['total_usd'].sum().reset_index()

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=total_rides['daily_date'], y=total_rides['rides'], name='Rides'))
            fig.add_trace(go.Scatter(x=total_usd['daily_date'], y=total_usd['total_usd'], name='USD'))
        else:
            # If "Merge Platforms" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by="platform", ascending=True)

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
        
            for platform in platforms:
                platform_data = filtered_data[filtered_data['platform'] == platform]
                platform_data_grouped = platform_data.groupby('daily_date')[['rides', 'total_usd']].sum().reset_index()
                fig.add_trace(go.Scatter(x=platform_data_grouped['daily_date'], y=platform_data_grouped['rides'], name=f'{platform} Rides'))
                fig.add_trace(go.Scatter(x=platform_data_grouped['daily_date'], y=platform_data_grouped['total_usd'], name=f'{platform} USD'))
    
        fig.update_layout(
                            title="Daily Rides - USD by Platform",
                            xaxis_title="Date",
                            yaxis_title="Count",
                            width=width_pixels,
                            height=height_pixels,
                         )
    
        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_completed_usd_daily.html')          
        
        fig.show()

    # Create an interactive widget to update the line plot
    widget = interactive(
        update_line_plot, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,        
        start_platform=start_platform_range_picker, 
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)
     


In [16]:
def display_completed_ratings():
    # Load the dataset
    df = pd.read_csv('../CSV/funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 4]
    
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range and age range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
            (df['daily_date'].dt.date >= start_date) &
            (df['daily_date'].dt.date <= end_date) &
            (df['age_range'] >= start_age_range) &
            (df['age_range'] <= end_age_range) &
            (df['platform'] >= start_platform) &
            (df['platform'] <= end_platform)
        ]
        return filtered_df
  
    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'

    platform_options = sorted(df['platform'].dropna().astype(str).unique())  
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=platform_options,
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=platform_options,
        value=default_end_platform_range,
        disabled=False
    )

    # Update function for the bar chart
    def update_bar_chart(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Merge" checkbox is checked, calculate the total users
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by=["age_range", "platform"], ascending=[True, True])

            mean_data = filtered_data.groupby(['platform'])['avg_rating'].mean().reset_index()
            mean_data['avg_rating']= mean_data['avg_rating'].round(2)

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()
            fig.add_trace(go.Bar(x=mean_data['platform'], y=mean_data['avg_rating'], name='USD'))
        else:
            # If "Merge" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by=["age_range", "platform"], ascending=[True, True])

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
            age_ranges = filtered_data['age_range'].unique()

            for age_range in age_ranges:
                age_range_data = filtered_data[filtered_data['age_range'] == age_range]
                ratings_by_platform = []

                for platform in platforms:
                    platform_data = age_range_data[age_range_data['platform'] == platform]
                    mean_rating = platform_data['avg_rating'].mean()
                    mean_rating = round(mean_rating, 2)
                    ratings_by_platform.append(mean_rating)

                # Create a label that includes both age_range and platform information
                label = [f'{age_range} - {platform}' for platform in platforms]

                fig.add_trace(go.Bar(x=label, y=ratings_by_platform, name=age_range))

        fig.update_layout(
                            title="Average Rating by Age Range - Platform",
                            xaxis_title="Age Range - Platform",
                            yaxis_title="Average Rating",
                            width=width_pixels,
                            height=height_pixels,            
                         )

        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_completed_ratings.html')          
        
        fig.show()
    # Create an interactive widget to update the bar chart
    widget = interactive(
        update_bar_chart, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,
        start_platform=start_platform_range_picker,
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)
 


In [17]:
# Create a function to display a plot
def display_completed_reviews_daily():
    # Load the dataset
    df = pd.read_csv('../CSV/funneldatawithdailydates.csv')
    df = df[df['funnel_step'] == 4]
    # Convert the 'daily_date' column to datetime
    df['daily_date'] = pd.to_datetime(df['daily_date'], format='%Y-%m-%d')

    # Filter the dataset based on the date range
    def filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform):
        start_date = pd.Timestamp(start_date).date()
        end_date = pd.Timestamp(end_date).date()
        filtered_df = df[
        (df['daily_date'].dt.date >= start_date) & 
        (df['daily_date'].dt.date <= end_date) &
        (df['age_range'] >= start_age_range) & 
        (df['age_range'] <= end_age_range) &        
        (df['platform'] >= start_platform) &
        (df['platform'] <= end_platform)
        ]
        return filtered_df

    # Filter out 'Unknown' values from the age_range column
    filtered_age_ranges = df[df['age_range'] != 'Unknown']['age_range']

    # Sort the unique age range values with a custom sorting function
    unique_age_ranges = sorted(filtered_age_ranges.unique(), key=lambda x: [int(age) for age in x.split('-')])
    # Append 'Unknown' to the end of the sorted list
    unique_age_ranges.append('Unknown')     

    # Define your default start and end age ranges
    default_start_age_range = '18-24'
    default_end_age_range = 'Unknown'
    
    start_age_range_picker = widgets.Dropdown(
        description='Start Age Range:',
        options=unique_age_ranges,
        value=default_start_age_range,
        disabled=False
    )
    end_age_range_picker = widgets.Dropdown(
        description='End Age Range:',
        options=unique_age_ranges,
        value=default_end_age_range,
        disabled=False
    )    
    # Define your default start and end platform ranges
    default_start_platform_range = 'android'
    default_end_platform_range = 'web'

    platform_options = sorted(df['platform'].dropna().astype(str).unique())  
    
    start_platform_range_picker = widgets.Dropdown(
        description='Start Platform:',
        options=platform_options,
        value=default_start_platform_range,
        disabled=False
    )
    end_platform_range_picker = widgets.Dropdown(
        description='End Platform:',
        options=platform_options,
        value=default_end_platform_range,
        disabled=False
    )

    # Update function for the line plot
    def update_line_plot(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform, total):
        if total:
            # If the "Merge Platforms" checkbox is checked, calculate the total rides and cancellations
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by="platform", ascending=True)
            total_reviews = filtered_data.groupby('daily_date')['total_reviews'].sum().reset_index()
            avg_rating = filtered_data.groupby('daily_date')['avg_rating'].mean().reset_index()

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=total_reviews['daily_date'], y=total_reviews['total_reviews'], name='Reviews'))
            fig.add_trace(go.Scatter(x=avg_rating['daily_date'], y=avg_rating['avg_rating'], name='Rating'))
        else:
            # If "Merge Platforms" is unchecked, display the data per platform
            filtered_data = filter_data(start_date, end_date, start_age_range, end_age_range, start_platform, end_platform)
            filtered_data = filtered_data.sort_values(by="platform", ascending=True)

            figsize_inches = (8.27, 5.87)
            dpi = 96
            width_pixels = int(figsize_inches[0] * dpi)
            height_pixels = int(figsize_inches[1] * dpi)
            
            fig = go.Figure()

            platforms = filtered_data['platform'].unique()
        
            for platform in platforms:
                platform_data = filtered_data[filtered_data['platform'] == platform]
                platform_data_reviews = platform_data.groupby('daily_date')['total_reviews'].sum().reset_index()
                platform_data_ratings = platform_data.groupby('daily_date')['avg_rating'].mean().reset_index()
                fig.add_trace(go.Scatter(x=platform_data_reviews['daily_date'], y=platform_data_reviews['total_reviews'], name=f'{platform} Reviews'))
                fig.add_trace(go.Scatter(x=platform_data_ratings['daily_date'], y=platform_data_ratings['avg_rating'], name=f'{platform} Rating'))
    
        fig.update_layout(
                            title="Daily Reviews - AVG.Rating per Platform",
                            xaxis_title="Date",
                            yaxis_title="Counts",
                            width=width_pixels,
                            height=height_pixels,            
                         )
    
        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_completed_reviews_daily.html')   
        
        fig.show()
        
    # Create an interactive widget to update the line plot
    widget = interactive(
        update_line_plot, 
        start_date=start_date_picker, 
        end_date=end_date_picker, 
        start_age_range=start_age_range_picker, 
        end_age_range=end_age_range_picker,        
        start_platform=start_platform_range_picker, 
        end_platform=end_platform_range_picker,
        total=total_checkbox
    )

    # Display the widgets
    display(widget)
    


In [18]:
def display_binary_users():
    # Load your dataset
    df = pd.read_csv('../CSV/user_rides_stats.csv')

    # Create a new column to label users as 'Full Cancellations', 'Full Completed Rides', or 'Mixed'
    df['User Type'] = 'Mixed'
    df.loc[(df['rides_completed'] == 0) & (df['rides_canceled'] > 0), 'User Type'] = 'Full Cancellations'
    df.loc[(df['rides_completed'] > 0) & (df['rides_canceled'] == 0), 'User Type'] = 'Full Completed Rides'

    # Create slider widgets for filtering
    rides_canceled_slider = widgets.FloatSlider(
        value=0,
        min=0,
        max=df['rides_canceled'].max(),
        step=1,
        description='Filter Rides Canceled:',
        style={'description_width': 'initial'},
        layout=Layout(width='50%')
    )

    rides_completed_slider = widgets.FloatSlider(
        value=0,
        min=0,
        max=df['rides_completed'].max(),
        step=1,
        description='Filter Rides Completed:',
        style={'description_width': 'initial'},
        layout=Layout(width='50%')
    )

    total_rides_slider = widgets.FloatSlider(
        value=0,
        min=0,
        max=df['total_rides'].max(),
        step=1,
        description='Filter Total Rides:',
        style={'description_width': 'initial'},
        layout=Layout(width='50%')
    )

    # Function to create the scatter plot using Plotly Graph Objects
    def create_scatter(rides_canceled_filter, rides_completed_filter, total_rides_filter):
        filtered_data = df[
            (df['rides_canceled'] >= rides_canceled_filter) &
            (df['rides_completed'] >= rides_completed_filter) &
            (df['total_rides'] >= total_rides_filter)
        ]

        figsize_inches = (8.27, 5.87)
        dpi = 96
        width_pixels = int(figsize_inches[0] * dpi)
        height_pixels = int(figsize_inches[1] * dpi)
        
        fig = go.Figure()

        # Add scatter traces for different user types
        user_types = filtered_data['User Type'].unique()
        colors = {'Mixed': 'gray', 'Full Cancellations': 'red', 'Full Completed Rides': 'blue'}

        for user_type in user_types:
            user_data = filtered_data[filtered_data['User Type'] == user_type]
            fig.add_trace(go.Scatter(
                x=user_data['user_id'],
                y=user_data['total_rides'],
                mode='markers',
                name=user_type,
                marker=dict(size=8, color=colors[user_type]),
                text=user_type + "<br>Rides Canceled: " + user_data['rides_canceled'].astype(str) +
                     "<br>Rides Completed: " + user_data['rides_completed'].astype(str)
            ))

        fig.update_layout(
            title='Full Cancellations vs. Full Completed Rides vs. Mixed',
            xaxis_title='User ID',
            yaxis_title='Rides',
            width=width_pixels,
            height=height_pixels,            
        )

        return fig

    # Function to update the scatter plot based on the slider values
    def update_scatter(rides_canceled_filter, rides_completed_filter, total_rides_filter):
        fig = create_scatter(rides_canceled_filter, rides_completed_filter, total_rides_filter)

        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_binary_users.html')        
        
        fig.show()

    # Create an interactive widget to update the scatter plot based on the slider values
    widget = interactive(
        update_scatter,
        rides_canceled_filter=rides_canceled_slider,
        rides_completed_filter=rides_completed_slider,
        total_rides_filter=total_rides_slider
    )

    # Display the widgets
    display(widget)


In [19]:
def display_charge_users_drivers():
    # Load the dataset from the CSV file
    df = pd.read_csv('../CSV/chargestatistics.csv')

    figsize_inches = (8.27, 5.87)
    dpi = 96
    width_pixels = int(figsize_inches[0] * dpi)
    height_pixels = int(figsize_inches[1] * dpi)
    
    # Create a grouped bar chart
    fig = go.Figure()

    # Define the unique charge_status values
    charge_statuses = df['charge_status'].unique()

    # Loop through each charge_status
    for charge_status in charge_statuses:
        charge_status_data = df[df['charge_status'] == charge_status]
        users_count = charge_status_data['users'].values[0]
        drivers_count = charge_status_data['drivers'].values[0]
        fig.add_trace(go.Bar(
            x=['Users', 'Drivers'],
            y=[users_count, drivers_count],
            name=charge_status,
            hoverinfo='x+y+name',
            text=[users_count, drivers_count],
            textposition='auto'
        ))

    # Update the layout
    fig.update_layout(
        title='Users VS Drivers',
        xaxis_title='Charge Status',
        yaxis_title='Count',
        barmode='group',
        width=width_pixels,
        height=height_pixels,        
    )
    # Transform the interactive Plotly chart into an Html and save it.
    fig.write_html('../InteractiveGraphics/interactive_charge_users_drivers.html')    
    # Show the chart
    fig.show()


In [20]:
def display_charge_rides_reviews():
    # Load the dataset from the CSV file
    df = pd.read_csv('../CSV/chargestatistics.csv')

    figsize_inches = (8.27, 5.87)
    dpi = 96
    width_pixels = int(figsize_inches[0] * dpi)
    height_pixels = int(figsize_inches[1] * dpi)
    
    # Create a grouped bar chart
    fig = go.Figure()

    # Define the unique charge_status values
    charge_statuses = df['charge_status'].unique()

    # Loop through each charge_status
    for charge_status in charge_statuses:
        charge_status_data = df[df['charge_status'] == charge_status]
        rides_count = charge_status_data['rides'].values[0]
        reviews_count = charge_status_data['reviews'].values[0]
        fig.add_trace(go.Bar(
            x=['Rides', 'Reviews'],
            y=[rides_count, reviews_count],
            name=charge_status,
            hoverinfo='x+y+name',          
            text=[rides_count, reviews_count],
            textposition='auto'
        ))

    # Update the layout
    fig.update_layout(
        title='Rides VS Reviews',
        xaxis_title='Charge Status',
        yaxis_title='Count',
        barmode='group',
        width=width_pixels,
        height=height_pixels,        
    )
    # Transform the interactive Plotly chart into an Html and save it.
    fig.write_html('../InteractiveGraphics/interactive_charge_rides_reviews.html')
    # Show the chart
    fig.show()


In [21]:
def display_charge_totalusd():
    # Load the dataset from the CSV file
    df = pd.read_csv('../CSV/chargestatistics.csv')

    figsize_inches = (8.27, 5.87)
    dpi = 96
    width_pixels = int(figsize_inches[0] * dpi)
    height_pixels = int(figsize_inches[1] * dpi)
    
    # Create a grouped bar chart
    fig = go.Figure()

    # Define the unique charge_status values
    charge_statuses = df['charge_status'].unique()

    # Loop through each charge_status
    for charge_status in charge_statuses:
        charge_status_data = df[df['charge_status'] == charge_status]
        totalusd = charge_status_data['total_usd'].values[0]
        fig.add_trace(go.Bar(
            x=[charge_status],
            y=[totalusd],
            name=charge_status,
            hoverinfo='x+y',        
            text=[totalusd],
            textposition='auto'
        ))

    # Update the layout
    fig.update_layout(
        title='Total USD by Charge Status',
        xaxis_title='Charge Status',
        yaxis_title='Total USD',
        barmode='group',
        width=width_pixels,
        height=height_pixels,        
    )
    # Transform the interactive Plotly chart into an Html and save it.
    fig.write_html('../InteractiveGraphics/interactive_charge_totalusd.html')
    # Show the chart
    fig.show()


In [22]:
def display_usd_users():
    # Load the dataset from the CSV file
    df = pd.read_csv('../CSV/user_declines.csv')

    # Create a slider widget for filtering lost_total_usd values
    lost_total_usd_slider = widgets.FloatSlider(
        value=0,  # Set the initial value to 0
        min=0,
        max=df['lost_total_usd'].max(),
        step=1,
        description='Lost Total USD Filter:',
        style={'description_width': 'initial'},
        layout=Layout(width='50%')
    )
    gained_total_usd_slider = widgets.FloatSlider(
        value=0,  # Set the initial value to 0
        min=0,
        max=df['gained_total_usd'].max(),
        step=1,
        description='Gained Total USD Filter:',
        style={'description_width': 'initial'},
        layout=Layout(width='50%')
    )

    # Create an output widget to display the chart
    output = Output()

    # Function to create the scatter plot using Plotly Graph Objects
    def create_usd_users_scatter(lost_total_usd_filter, gained_total_usd_filter):
        # Filter the data based on the slider value
        filtered_data = df[(df['lost_total_usd'] >= lost_total_usd_filter) & (df['gained_total_usd'] >= gained_total_usd_filter)]

        figsize_inches = (8.27, 5.87)
        dpi = 96
        width_pixels = int(figsize_inches[0] * dpi)
        height_pixels = int(figsize_inches[1] * dpi)
        
        # Create a scatter plot using Plotly Graph Objects
        fig = go.Figure()

        # Add scatter traces for lost_total_usd and gained_total_usd
        fig.add_trace(go.Scatter(
            x=filtered_data['lost_total_usd'],
            y=filtered_data['user_id'],
            mode='markers',
            marker=dict(size=10, color='red'),  # Customize marker color
            name='Lost Total USD'
        ))

        fig.add_trace(go.Scatter(
            x=filtered_data['gained_total_usd'],
            y=filtered_data['user_id'],
            mode='markers',
            marker=dict(size=10, color='blue'),  # Customize marker color
            name='Gained Total USD'
        ))

        fig.update_layout(
            title='Lost & Gained Total USD by Users',
            xaxis_title='Total USD',
            yaxis_title='User ID',
            width=width_pixels,
            height=height_pixels,            
        )

        return fig

    # Function to update the scatter plot based on the slider value
    def update_usd_users_scatter(lost_total_usd_filter, gained_total_usd_filter):
        # Clear the previous output
        with output:
            output.clear_output()

        # Get the Plotly figure
        fig = create_usd_users_scatter(lost_total_usd_filter, gained_total_usd_filter)

        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_usd_users.html')            
        
        # Show the scatter plot
        fig.show()

    # Create an interactive widget to update the scatter plot based on the slider value
    widget = interactive(update_usd_users_scatter, lost_total_usd_filter=lost_total_usd_slider, gained_total_usd_filter=gained_total_usd_slider)

    # Display the widgets
    display(widget)


In [23]:
def display_usd_drivers():
    # Load the dataset from the CSV file
    df = pd.read_csv('../CSV/driver_declines.csv')

    # Create a slider widget for filtering lost_total_usd values
    lost_total_usd_slider = widgets.FloatSlider(
        value=0,  # Set the initial value to 0
        min=0,
        max=df['lost_total_usd'].max(),
        step=1,
        description='Lost Total USD Filter:',
        style={'description_width': 'initial'},
        layout=Layout(width='50%')
    )
    gained_total_usd_slider = widgets.FloatSlider(
        value=0,  # Set the initial value to 0
        min=0,
        max=df['gained_total_usd'].max(),
        step=1,
        description='Gained Total USD Filter:',
        style={'description_width': 'initial'},
        layout=Layout(width='50%')
    )

    # Create an output widget to display the chart
    output = Output()

    # Function to create the scatter plot using Plotly Graph Objects
    def create_usd_drivers_scatter(lost_total_usd_filter, gained_total_usd_filter):
        # Filter the data based on the slider value
        filtered_data = df[(df['lost_total_usd'] >= lost_total_usd_filter) & (df['gained_total_usd'] >= gained_total_usd_filter)]

        figsize_inches = (8.27, 5.87)
        dpi = 96
        width_pixels = int(figsize_inches[0] * dpi)
        height_pixels = int(figsize_inches[1] * dpi)
        
        # Create a scatter plot using Plotly Graph Objects
        fig = go.Figure()

        # Add scatter traces for lost_total_usd and gained_total_usd
        fig.add_trace(go.Scatter(
            x=filtered_data['lost_total_usd'],
            y=filtered_data['driver_id'],
            mode='markers',
            marker=dict(size=10, color='red'),  # Customize marker color
            name='Lost Total USD'
        ))

        fig.add_trace(go.Scatter(
            x=filtered_data['gained_total_usd'],
            y=filtered_data['driver_id'],
            mode='markers',
            marker=dict(size=10, color='blue'),  # Customize marker color
            name='Gained Total USD'
        ))

        fig.update_layout(
            title='Lost & Gained Total USD by Drivers',
            xaxis_title='Total USD',
            yaxis_title='Driver ID',
            width=width_pixels,
            height=height_pixels,
        )

        return fig

    # Function to update the scatter plot based on the slider value
    def update_usd_drivers_scatter(lost_total_usd_filter, gained_total_usd_filter):
        # Clear the previous output
        with output:
            output.clear_output()

        # Get the Plotly figure
        fig = create_usd_drivers_scatter(lost_total_usd_filter, gained_total_usd_filter)

        # Transform the interactive Plotly chart into an Html and save it.
        fig.write_html('../InteractiveGraphics/interactive_usd_drivers.html') 
        
        # Show the scatter plot
        fig.show()

    # Create an interactive widget to update the scatter plot based on the slider value
    widget = interactive(update_usd_drivers_scatter, lost_total_usd_filter=lost_total_usd_slider, gained_total_usd_filter=gained_total_usd_slider)

    # Display the widgets
    display(widget)
      

In [24]:
# Define the main categories and their respective subcategories
categories = {
    'Funnel': ['User Funnel', 'Rides Funnel'],
    'Downloads': ['Daily Downloads per Platform'],
    'Signups': [
                'Signups by Age Range - Platform',
                'Daily Signups per Platform'
               ],
    'Rides Requested': [
                        'Ride Requested Users by Age Range - Platform',
                        'Ride Requests by Age Range - Platform',
                        'Daily Rides & Cancellations per Platform',
                        'Surge-Pricing', 'Minimum Rides Per User'
                       ],
    'Rides Completed': [
                        'Rides Completed Users by Age Range - Platform',
                        'Rides Completed by Age Range - Platform',
                        'USD by Age Range - Platform',
                        'Daily Rides - USD per Platform',
                        'Average Rating by Age Range - Platform',
                        'Daily Reviews - AVG.Rating per Platform',
                        'Binary User Behaviour',
                       ],
    'Charge Status':[
                     'Users VS Drivers',
                     'Rides VS Reviews',
                     'Total USD by Charge Status',
                     'Lost & Gained Total USD by Users',
                     'Lost & Gained Total USD by Drivers',
                    ],
}

# Create dropdown widgets for main categories and subcategories
main_category_selector = widgets.Dropdown(
    options=categories.keys(),
    description='Main Category:'
)

subcategory_selector = widgets.Dropdown(
    options=categories[main_category_selector.value],
    description='Subcategory:'
)

# Function to update subcategories based on the selected main category
def update_subcategories(change):
    subcategory_selector.options = categories[change.new]

main_category_selector.observe(update_subcategories, names='value')

#Set default visualization for loading

def set_default_visualization():
    selected_visualization = 'User Funnel'
    display_visualization(selected_visualization)
   

# Display the selected visualization based on the chosen subcategory
def display_visualization(selected_visualization):
    with output:
        output.clear_output()
        if selected_visualization == 'User Funnel':
            display_funnel_users_visualization()
        elif selected_visualization == 'Rides Funnel':
            display_funnel_rides_visualization()
        elif selected_visualization == 'Daily Downloads per Platform':
            display_download_platform()
        elif selected_visualization == 'Signups by Age Range - Platform':
            display_signups_visualization()
        elif selected_visualization == 'Daily Signups per Platform':
            display_signupsdaily_visualization()
        elif selected_visualization == 'Ride Requested Users by Age Range - Platform':
            display_user_by_riderequests_visualization()
        elif selected_visualization == 'Ride Requests by Age Range - Platform':
            display_riderequests_visualization()
        elif selected_visualization == 'Daily Rides & Cancellations per Platform':
            display_rides_cancellations_visualization()
        elif selected_visualization == 'Surge-Pricing':
            display_surge_pricing()
        elif selected_visualization == 'Minimum Rides Per User':
            display_minridesperuser()            
        elif selected_visualization == 'Rides Completed Users by Age Range - Platform':
            display_user_by_ridecompletes_visualization()         
        elif selected_visualization == 'Rides Completed by Age Range - Platform':
            display_completed_age_platform()
        elif selected_visualization == 'USD by Age Range - Platform':
            display_completed_usd()            
        elif selected_visualization == 'Daily Rides - USD per Platform':
            display_completed_usd_daily()            
        elif selected_visualization == 'Average Rating by Age Range - Platform':
            display_completed_ratings()
        elif selected_visualization == 'Daily Reviews - AVG.Rating per Platform':
            display_completed_reviews_daily()
        elif selected_visualization == 'Binary User Behaviour':
            display_binary_users()
        elif selected_visualization == 'Users VS Drivers':
            display_charge_users_drivers()
        elif selected_visualization == 'Rides VS Reviews':
            display_charge_rides_reviews()
        elif selected_visualization == 'Total USD by Charge Status':
            display_charge_totalusd()
        elif selected_visualization == 'Lost & Gained Total USD by Users':
            display_usd_users() 
        elif selected_visualization == 'Lost & Gained Total USD by Drivers':
            display_usd_drivers()


# Set up the event handler for the subcategory value change
widgets.interactive(display_visualization, selected_visualization=subcategory_selector)

# Create an output widget for displaying the visualizations
output = widgets.Output()

# Display the main category and subcategory dropdowns, along with the output widget
display(main_category_selector, subcategory_selector, output)
set_default_visualization()

Dropdown(description='Main Category:', options=('Funnel', 'Downloads', 'Signups', 'Rides Requested', 'Rides Co…

Dropdown(description='Subcategory:', options=('User Funnel', 'Rides Funnel'), value='User Funnel')

Output()